In [1]:
%tensorflow_version 2.x
from tensorflow import keras
import numpy as np
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

TensorFlow 2.x selected.


In [0]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [0]:
train_images = train_images / 255.0
test_images = test_images / 255.0

In [0]:
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)

In [0]:
y_train = to_categorical(train_labels, 10)
y_test = to_categorical(test_labels, 10)

In [0]:
model = keras.Sequential([
                          keras.layers.Conv2D(32, (3,3), padding='same', activation='relu', kernel_initializer='he_uniform', kernel_regularizer=keras.regularizers.l2(0.001), input_shape  = (28, 28, 1)),
                          # keras.layers.BatchNormalization(),
                          keras.layers.MaxPooling2D((2,2)),

                          keras.layers.Conv2D(64, (3,3), padding='same', activation='relu', kernel_initializer='he_uniform', kernel_regularizer=keras.regularizers.l2(0.001)),
                          # keras.layers.BatchNormalization(),
                          keras.layers.MaxPooling2D((2,2)),

                          keras.layers.Conv2D(128, (3,3), padding='same', activation='relu', kernel_initializer='he_uniform', kernel_regularizer=keras.regularizers.l2(0.001)),
                          # keras.layers.BatchNormalization(),
                          keras.layers.MaxPooling2D((2,2)),

                          keras.layers.Flatten(),
                          keras.layers.Dense(1000, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=keras.regularizers.l2(0.001)),
                          keras.layers.Dense(10, activation='softmax')
])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 128)         73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0

In [0]:
class myCallback(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs = {}):
    if (logs.get('loss') < 0.01):
      print('Loss is Lower, Cancelling Training')
      self.model.stop_training = True


callback = myCallback()

# OR You Can Use EarlyStopping from tf.keras.callbacks.EarlyStopping

my_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=9, mode='min', restore_best_weights=True)

In [0]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [10]:
model.fit(train_images, train_labels, epochs=100, callbacks=[my_callback], validation_split=0.2, batch_size=256)

Train on 48000 samples, validate on 12000 samples
Epoch 1/100
48000/48000 [==============================] - 4s 78us/sample - loss: 1.9130 - acc: 0.8090 - val_loss: 1.2753 - val_acc: 0.8670
Epoch 2/100
48000/48000 [==============================] - 2s 36us/sample - loss: 1.0437 - acc: 0.8811 - val_loss: 0.9071 - val_acc: 0.8763
Epoch 3/100
48000/48000 [==============================] - 2s 35us/sample - loss: 0.7725 - acc: 0.8922 - val_loss: 0.7043 - val_acc: 0.8936
Epoch 4/100
48000/48000 [==============================] - 2s 34us/sample - loss: 0.6323 - acc: 0.9000 - val_loss: 0.6156 - val_acc: 0.8915
Epoch 5/100
48000/48000 [==============================] - 2s 35us/sample - loss: 0.5490 - acc: 0.9050 - val_loss: 0.5518 - val_acc: 0.8950
Epoch 6/100
48000/48000 [==============================] - 2s 35us/sample - loss: 0.4938 - acc: 0.9119 - val_loss: 0.5321 - val_acc: 0.8935
Epoch 7/100
48000/48000 [==============================] - 2s 35us/sample - loss: 0.4551 - acc: 0.9149 - val_l

In [11]:
loss, acc = model.evaluate(train_images, train_labels, verbose=0)
print(f'Training Loss: {loss:.3}, \t Training Accuracy: {acc:.3}' )
loss, acc = model.evaluate(test_images, test_labels, verbose=0)
print(f'Testing Loss: {loss:.3}, \t Testing Accuracy: {acc:.3}')

Training Loss: 0.258, 	 Training Accuracy: 0.956
Testing Loss: 0.362, 	 Testing Accuracy: 0.915


In [0]:
json = model.to_json()

with open('model.json', 'w') as json_f:
  json_f.write(json)

yaml = model.to_yaml()

with open('model.yaml', 'w') as yaml_f:
  yaml_f.write(yaml)

In [0]:
model.save('Keras_Model.h5')